In [194]:
# Import Libraries
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt

In [195]:
#Import Data
url="https://raw.githubusercontent.com/arwhit/rental_price_data/main/Annual_Rent_Data_from_JAN17_JAN22.csv"
rent_data=pd.read_csv(url)
#rent_data.rename(columns = {'Jan-21':'JAN2021_Rent', 'Jan-22':'JAN2022_Rent','%YOY':"Percent_Change"}, inplace = True)

In [203]:
#read data, make neccesary calculations, and create the dataframe
#this dataset has 99 rows and 19 columns
City=[]
State=[]
Region=[]
Country=[]
Date=[]
AVG_Date=[]
Rent_Price=[]
D_Change=[]
P_Change=[]

#for every row in the data frame
for i in range(len(rent_data.index)):
    #reset year
    year=2017
    #for every needed column in the dataframe
    for j in range(3,9):
            date=str(year)
            #copy neccesary data
            City.append(rent_data.iloc[i,0])
            State.append(rent_data.iloc[i,1])
            Region.append(rent_data.iloc[i,2])
            Country.append('US')
            #case for first year
            if j == 3:
                Date.append(date)
                Rent_Price.append(rent_data.iloc[i,j])
                D_Change.append(float('nan'))
                P_Change.append(float('nan'))
                AVG_Date.append(float('nan'))
            else:
                Date.append(date)
                AVG_Date.append(str(year-1))
                Rent_Price.append(rent_data.iloc[i,j])
                #calculate annual change as dollar value
                D_Change.append(rent_data.iloc[i,j]-rent_data.iloc[i,(j-1)])
                #calculate annual change as percent
                P_Change.append(100*(rent_data.iloc[i,j]-rent_data.iloc[i,(j-1)])/rent_data.iloc[i,(j-1)])
            #adjust year
            year += 1
            
#create dictionary of independent lists
data={'City': City,
     'State': State,
     'Region': Region,
     'Country': Country,
     'Date':Date, #Note: this is the raw date
      'Year': AVG_Date, #Note: this is the year for the averages calcualted
     'Rent Price': Rent_Price,
     'Dollar Change': D_Change,
     'Percent Change': P_Change}
#convert to dataframe
clean_data=pd.DataFrame(data)

In [204]:
#Group the data by region and date for easier calculations and visualizations and preview
reg_data=clean_data.groupby(['Region', 'Date']).mean()

In [215]:
#Create Graph For average Rent Increase based on the major Metro Areas of Each State
dropdown=alt.binding_select(options=clean_data['State'].unique(), name="Select a State")
selection=alt.selection(type="single", fields=['State'], bind=dropdown)

V1=alt.Chart(clean_data).mark_line().encode(
    x='Date',
    y=alt.Y('mean(Rent Price):Q'),
    color=alt.Color('State', legend=None),
    #tooltip=['State','mean(Rent_Price)'],
    opacity=alt.condition(selection,alt.value(1),alt.value(0))
).add_selection(selection).properties(width=400, height=400, title='Average Rent Price By Year (Select State in Dropdown)')

V1

alt.Chart(...)

In [213]:
#Simple Plot of Percent Rent Increase over time by region and total

#Create Selection
selection = alt.selection(type='multi', fields=['Region'], bind='legend')

base = alt.Chart(clean_data).mark_circle().encode(
    x=alt.X('Year:N', 
            axis=alt.Axis(title="Year")),
    y=alt.Y('Percent Change:Q', 
            axis=alt.Axis(title="% Rent Change")),
    color='Region:N',
    size='Rent Price',
    tooltip=['City','State','Percent Change','Rent Price'],
    opacity=alt.condition(selection, alt.value(1),alt.value(0.1))
).add_selection(selection).properties(width=400, height=400, title='Percent Rent Change by Year and Region (click region in legend, click in white space to reset)')

base

alt.Chart(...)

In [214]:
#binned data
selection = alt.selection(type='multi', fields=['Year'], bind='legend')

alt.Chart(clean_data).mark_circle().encode(
    alt.X('Dollar Change:Q', bin=True),
    alt.Y('Percent Change:Q', bin=True),
    size='count()',
    color='Year',
    #tooltip=['Region'],
    opacity=alt.condition(selection, alt.value(1),alt.value(0.1))
).add_selection(selection).properties(title='Percent Change vs. Dollar Change by Year (Click Year in Legend, Click White to Reset)')

alt.Chart(...)